In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-tachygraphy/Tachygraphy_EmotionMoodtags_Dataset.csv
/kaggle/input/dataset-tachygraphy/Tachygraphy_MicroText-AIO-V3.xlsx
/kaggle/input/dataset-tachygraphy/Tachygraphy_dataset_main.csv
/kaggle/input/dataset-tachygraphy/Tachygraphy_MicroText-AIO-V2.xlsx


In [2]:
dataset = pd.read_excel('/kaggle/input/dataset-tachygraphy/Tachygraphy_MicroText-AIO-V3.xlsx')

In [3]:
df = dataset

In [4]:
df.rename(columns={'Informal Text':'input', 'Expanded Meaning':'target'}, inplace = True)

In [5]:
df.head()

,input,target
0,"omg, JEE prep is killing me rn","Oh my god, Joint Entrance Examination preparat..."
1,u up 4 a break b4 UPSC revision?,Are you up for a break before Union Public Ser...
2,"ttyl, finishing da CAT mock","Talk to you later, finishing the Common Admiss..."
3,"nah, dat GATE paper was brutal af","No, that Graduate Aptitude Test in Engineering..."
4,sup? u done w/ ur IIT assignment?,What's up? Are you done with your Indian Insti...


In [6]:
df['input'] = df['input'].astype(str)
df['target'] = df['target'].astype(str)

In [7]:
import torch
from torch.utils.data import DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BertModel, BertTokenizer
# from torch.cuda.amp import autocast, GradScaler
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
import ray
from ray import tune
import torch.nn as nn

from torch.optim import AdamW
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
from sklearn.model_selection import train_test_split
import optuna
# from optuna.integration import PyTorchLightningPruner
from ray import tune
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import ASHAScheduler
# from ray.tune.integration.pytorch import TuneReportCallback
from torch.amp import GradScaler, autocast
# from ray.tune.integration.optuna import OptunaSearch
from ray.tune.search.optuna import OptunaSearch
from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch
from torch import autocast
# from ray import tune
# from ray.tune.integration.tensorboard import TensorBoardReporter
from ray.tune.logger import TBXLogger
from torch.utils.tensorboard import SummaryWriter
from ray.train import report
# from ray.tune.integration.jupyter import JupyterNotebookReporter
from ray.tune import JupyterNotebookReporter
# from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
from ray.tune.schedulers import HyperBandScheduler, AsyncHyperBandScheduler


import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings


import emoji
from bs4 import BeautifulSoup
import os
import re
import string
import json


import argparse # CPMP

import scipy as sp

from tqdm.auto import tqdm

In [8]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
#     text = emoji.demojize(text)
#     text = re.sub(r'\:(.*?)\:','',text)
#     text = str(text).lower()    #Making Text Lowercase
#     text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
#     text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, 0-9, "%", ".", "&", ",", "'", "?", "!", ",", "'", ";", "-")
    text = re.sub(r"[^a-zA-Z0-9?.!,¿'%&,';-]+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
#     text = clean_contractions(text, contraction_mapping)
    text = clean_text(text)
#     text = clean_contractions(text, contraction_mapping)
#     text = clean_special_chars(text, punct, punct_mapping)
#     text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [9]:
df['input'] = df['input'].apply(lambda x: text_preprocessing_pipeline(x))
df['target'] = df['target'].apply(lambda x: text_preprocessing_pipeline(x))

<ipython-input-8-93bb6d8e489f>:51: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'lxml').get_text()


In [10]:
df['input'] = df['input'].astype(str)
df['target'] = df['target'].astype(str)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
# class TextDataset(Dataset):
#     def __init__(self, tokenizer, texts, targets, max_length=256):
#         self.tokenizer = tokenizer
#         self.texts = texts
#         self.targets = targets
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = self.texts[idx]
#         target = self.targets[idx]
#         encodings = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length)
#         target_encodings = self.tokenizer(target, truncation=True, padding='max_length', max_length=self.max_length)

#         return {
#             'input_ids': torch.tensor(encodings['input_ids'], dtype=torch.long),
#             'attention_mask': torch.tensor(encodings['attention_mask'], dtype=torch.long),
#             'labels': torch.tensor(target_encodings['input_ids'], dtype=torch.long)
#         }

In [13]:
from torch.utils.data import Dataset
import torch

class TextDataset(Dataset):
    def __init__(self, tokenizer, dataset, max_length=256):
        self.tokenizer = tokenizer
        self.texts = dataset['input']
        self.targets = dataset['target']
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        target = self.targets[idx]
        encodings = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length)
        target_encodings = self.tokenizer(target, truncation=True, padding='max_length', max_length=self.max_length)

        return {
            'input_ids': torch.tensor(encodings['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encodings['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(target_encodings['input_ids'], dtype=torch.long)
        }


In [14]:
def test_train_split(dataset, test_ratio=0.1):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


In [15]:
train_ds_pd, validation_ds_pd = test_train_split(df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(validation_ds_pd)))

9214 examples in training, 1066 examples in testing.


In [16]:
train_ds_pd = train_ds_pd.reset_index(drop=True)
validation_ds_pd = validation_ds_pd.reset_index(drop=True)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
# # Function to calculate cosine similarity between predicted and actual sentences
# def calculate_cosine_similarity(pred_text, target_text, bert_model, bert_tokenizer, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
#     # Move model to GPU if available
#     bert_model.to(device)

#     # Tokenize and encode both texts, moving inputs to the GPU
#     pred_inputs = bert_tokenizer(pred_text, return_tensors="pt", truncation=True, padding=True).to(device)
#     target_inputs = bert_tokenizer(target_text, return_tensors="pt", truncation=True, padding=True).to(device)
    
#     # Get embeddings from BERT
#     with torch.no_grad():
#         with autocast(device_type=device.type):
#             pred_embeddings = bert_model(**pred_inputs).last_hidden_state.mean(dim=1).to(device)
#             target_embeddings = bert_model(**target_inputs).last_hidden_state.mean(dim=1).to(device)

#     # Calculate cosine similarity
#     similarity = cosine_similarity(pred_embeddings.cpu().numpy(), target_embeddings.cpu().numpy())[0][0]
#     return similarity

In [19]:
def calculate_cosine_similarity(pred_text, target_text, model, tokenizer, device, model_type="bart"):
    # Move model to GPU if available
    model.to(device)

    # Tokenize and encode both texts, moving inputs to the GPU
    pred_inputs = tokenizer(pred_text, return_tensors="pt", truncation=True, padding=True).to(device)
    target_inputs = tokenizer(target_text, return_tensors="pt", truncation=True, padding=True).to(device)

    # Get embeddings based on the model type
    with torch.no_grad():
#         with torch.cuda.amp.autocast(enabled=True):  # Autocast for mixed precision training/inference
        if model_type in ["bart", "pegasus", "t5"]:  # For seq2seq models like BART, Pegasus, T5
            # Use encoder outputs
            pred_embeddings = model.model.encoder(input_ids=pred_inputs.input_ids, attention_mask=pred_inputs.attention_mask).last_hidden_state.mean(dim=1).to(device)
            target_embeddings = model.model.encoder(input_ids=target_inputs.input_ids, attention_mask=target_inputs.attention_mask).last_hidden_state.mean(dim=1).to(device)

        elif model_type == "llama":  # For LLaMA models
            pred_embeddings = model(**pred_inputs).hidden_states[-1].mean(dim=1).to(device)
            target_embeddings = model(**target_inputs).hidden_states[-1].mean(dim=1).to(device)

        else:  # For models like BERT, RoBERTa (directly exposing `last_hidden_state`)
            pred_embeddings = model(**pred_inputs).last_hidden_state.mean(dim=1).to(device)
            target_embeddings = model(**target_inputs).last_hidden_state.mean(dim=1).to(device)

    # Calculate cosine similarity
    similarity = cosine_similarity(pred_embeddings.cpu(), target_embeddings.cpu())[0].item()
    return similarity

In [20]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        """
        EarlyStopping to stop training when a metric has stopped improving.

        Args:
            patience (int): How long to wait after last time validation loss improved.
            verbose (bool): If True, prints a message for each validation loss improvement.
            delta (float): Minimum change to qualify as an improvement.
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.counter = 0
        self.best_loss = np.inf
        self.early_stop = False

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
#             if self.verbose:
#                 print(f'Validation loss improved to {val_loss:.4f}')
#             torch.save(model.state_dict(), 'checkpoint.pt')
        else:
            self.counter += 1
#             if self.verbose:
#                 print(f'Validation loss did not improve. Patience: {self.patience}, Counter: {self.counter}')
        
        if self.counter >= self.patience:
            self.early_stop = True



In [21]:
# def train_bart_model(config, checkpoint_dir=None):
#     tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
#     model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
    
    
#     train_texts = train_ds_pd['input']
#     train_labels = train_ds_pd['target']
#     val_texts = validation_ds_pd['input']
#     val_labels = validation_ds_pd['target']
    
#     train_dataset = TextDataset(tokenizer, train_texts, train_labels)
#     val_dataset = TextDataset(tokenizer, val_texts, val_labels)
    
#     train_dataloader = DataLoader(train_dataset, batch_size=int(config['batch_size']), shuffle=True)
#     val_dataloader = DataLoader(val_dataset, batch_size=int(config['batch_size']))

#     optimizer = AdamW(model.parameters(), lr=config['lr'])

#     # Training loop
#     model.train()
#     for epoch in range(int(config['epochs'])):
#         epoch_loss = 0
#         for batch in train_dataloader:
#             optimizer.zero_grad()
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
            
#             outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs.loss
#             loss.backward()
#             optimizer.step()
#             epoch_loss += loss.item()
        
#         # Validation loop
#         model.eval()
#         val_loss = 0
#         with torch.no_grad():
#             for batch in val_dataloader:
#                 input_ids = batch['input_ids'].to(device)
#                 attention_mask = batch['attention_mask'].to(device)
#                 labels = batch['labels'].to(device)
#                 outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#                 val_loss += outputs.loss.item()

#         # Report metrics to Ray Tune
#         tune.report(train_loss=epoch_loss, val_loss=val_loss)

In [22]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
# bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-base')
# DebertaV2Model.from_pretrained('microsoft/deberta-v3-base')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [23]:
# # def train_bart_model_with_mixed_precision(config, checkpoint_dir=None):

# # config, train_dataset, val_dataset, device
# def train_model(config, train_dataset, val_dataset, device):
#     # tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
#     tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
#     # model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
#     model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
    
# #     train_texts = train_ds_pd['input']
# #     train_labels = train_ds_pd['target']
# #     val_texts = validation_ds_pd['input']
# #     val_labels = validation_ds_pd['target']

# #     train_dataset = TextDataset(tokenizer, train_texts, train_labels)
# #     val_dataset = TextDataset(tokenizer, val_texts, val_labels)
    
#     train_loader = DataLoader(train_dataset, batch_size=int(config['batch_size']), shuffle=False)
#     val_loader = DataLoader(val_dataset, batch_size=int(config['batch_size']), shuffle=False)

#     optimizer = AdamW(model.parameters(), lr=config['lr'])
#     scaler = GradScaler()  # Initialize the gradient scaler for mixed precision
    
#     scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
    
#     early_stopping = EarlyStopping(patience=3, verbose=True)

#     # Training loop with mixed precision
#     for epoch in range(int(config['epochs'])):
#         model.train()
        
#         total_train_loss = 0.0
        
#         for batch in train_loader:
            
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
            
#             optimizer.zero_grad()
            
#             with autocast(device_type=device.type):  # Use autocast for mixed precision
#                 outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#                 loss = outputs.loss

#             # Scale the loss and backpropagate
#             scaler.scale(loss).backward()
#             scaler.step(optimizer)
#             scaler.update()

#             total_train_loss += loss.item()
            
#         avg_train_loss = total_train_loss / len(train_loader)

#         # Validation loop with mixed precision
#         model.eval()
        
#         total_val_loss = 0.0
#         val_similarity = 0.0
#         num_val_samples = 0
        
#         with torch.no_grad():
#             for batch in val_loader:
#                 input_ids = batch['input_ids'].to(device)
#                 attention_mask = batch['attention_mask'].to(device)
#                 labels = batch['labels'].to(device)
                
#                 with autocast(device_type=device.type):
#                     outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#                     loss = outputs.loss

#                 preds = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=512)
#                 pred_texts = tokenizer.batch_decode(preds, skip_special_tokens=True)
#                 target_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

#                 for pred_text, target_text in zip(pred_texts, target_texts):
#                     similarity = calculate_cosine_similarity(pred_text, target_text, model, tokenizer, device)
#                     val_similarity += similarity
#                     num_val_samples += 1

#                 total_val_loss += loss.item()
                
#         avg_val_loss = total_val_loss / len(val_loader)
# #         avg_val_similarity = val_similarity / len(val_loader)
#         avg_val_similarity = val_similarity / num_val_samples
        


#         # Report both loss and similarity
# #         tune.report(train_loss=avg_train_loss, val_loss=avg_val_loss, val_similarity=avg_val_similarity)
        
#         report({
#             "loss": avg_val_loss,
#             "similarity": avg_val_similarity,
#             "train_loss": avg_train_loss,
#             "early_stopping_epoch": epoch + 1,
#         })
        
        
#         # Learning rate scheduler step
#         scheduler.step(avg_val_loss)
        
        
#         early_stopping(avg_val_loss, model)
#         if early_stopping.early_stop:
# #             print("Early stopping")
#             break
    
#     return model
    
    
# def train_fn(config):
#     # Load your data here
#     tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
# #     bert_tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-base')

# #     train_dataset = DisasterDataset(train_ds_pd, tokenizer, max_len=192)
# #     val_dataset = DisasterDataset(validation_ds_pd, tokenizer, max_len=192)

#     train_dataset = TextDataset(tokenizer, train_ds_pd)
#     val_dataset = TextDataset(tokenizer, validation_ds_pd)

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     train_model(config, train_dataset, val_dataset, device)
    


In [24]:
# import torch
# import torch.optim as optim
# from torch.utils.data import DataLoader
# from torch.optim import AdamW
# from torch.cuda.amp import GradScaler, autocast
# from transformers import AutoTokenizer, AutoModelForCausalLM

# def train_model(config, train_dataset, val_dataset, device):
#     """
#     Train the LLaMA 3 8B model using full fine-tuning with mixed precision.
    
#     Args:
#         config (dict): Training configuration containing batch_size, lr, and epochs.
#         train_dataset (Dataset): Training dataset.
#         val_dataset (Dataset): Validation dataset.
#         device (torch.device): CUDA or CPU device.
        
#     Returns:
#         model: Trained model.
#     """

#     # Load tokenizer and model
#     model_name = "meta-llama/Meta-Llama-3-8B"
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForCausalLM.from_pretrained(model_name)

#     model.to(device)

#     # DataLoaders
#     train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

#     # Optimizer, LR Scheduler, and Mixed Precision
#     optimizer = AdamW(model.parameters(), lr=config['lr'])
#     scaler = GradScaler()
#     scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

#     early_stopping = EarlyStopping(patience=3, verbose=True)

#     for epoch in range(config['epochs']):
#         model.train()
#         total_train_loss = 0.0

#         for batch in train_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)

#             optimizer.zero_grad()

#             with autocast(device_type=device.type):
#                 outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#                 loss = outputs.loss

#             scaler.scale(loss).backward()
#             scaler.step(optimizer)
#             scaler.update()

#             total_train_loss += loss.item()

#         avg_train_loss = total_train_loss / len(train_loader)

#         # Validation
#         model.eval()
#         total_val_loss = 0.0
#         val_similarity = 0.0
#         num_val_samples = 0

#         with torch.no_grad():
#             for batch in val_loader:
#                 input_ids = batch['input_ids'].to(device)
#                 attention_mask = batch['attention_mask'].to(device)
#                 labels = batch['labels'].to(device)

#                 with autocast(device_type=device.type):
#                     outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#                     loss = outputs.loss

#                 preds = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=512)
#                 pred_texts = tokenizer.batch_decode(preds, skip_special_tokens=True)
#                 target_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

#                 for pred_text, target_text in zip(pred_texts, target_texts):
#                     similarity = calculate_cosine_similarity(pred_text, target_text, model, tokenizer, device)
#                     val_similarity += similarity
#                     num_val_samples += 1

#                 total_val_loss += loss.item()

#         avg_val_loss = total_val_loss / len(val_loader)
#         avg_val_similarity = val_similarity / num_val_samples

#         # Logging metrics
#         report({
#             "loss": avg_val_loss,
#             "similarity": avg_val_similarity,
#             "train_loss": avg_train_loss,
#             "early_stopping_epoch": epoch + 1,
#         })

#         # Learning rate scheduling & early stopping
#         scheduler.step(avg_val_loss)
#         early_stopping(avg_val_loss, model)

#         if early_stopping.early_stop:
#             print("Early stopping triggered.")
#             break

#     return model


In [25]:
# # Hyperparameter tuning function with Ray Tune
# def tune_model_with_mixed_precision():
#     search_space = {
#         "lr": tune.loguniform(1e-7, 1e-2),
#         "batch_size": tune.choice([8, 16, 32]),
#         "epochs": tune.choice([3, 5, 7, 10, 12, 15, 20])
#     }
    
#     reporter = JupyterNotebookReporter(
#         metric_columns=["loss", "training_iteration", "train_loss", "similarity", "early_stopping_epoch"],
#         print_intermediate_tables=False,
#     )
    
#     optuna_search = OptunaSearch(
#         metric="loss",
#         mode="min"
#     )
    
    

#     scheduler = ASHAScheduler(
#         metric='loss',
#         mode='min',
#         max_t=22,
#         grace_period=3,
#         reduction_factor=2
#     )



#     analysis = tune.run(
#         train_fn,
#         config=search_space,
#         num_samples=30,
#         resources_per_trial={"cpu": 2, "gpu": 1},
#         progress_reporter=reporter,
#         search_alg=optuna_search
#     )

#     print("Best hyperparameters found:", analysis.best_config)
    
#     best_trial = result.get_best_trial("accuracy", "max", "last")

In [26]:
# ray.init(ignore_reinit_error=True)
# tune_model_with_mixed_precision()

In [27]:
# # search_space = {
# #     "lr": tune.loguniform(1e-7, 1e-2),
# #     "batch_size": tune.choice([8, 16, 32]),
# #     "epochs": tune.choice([3, 5, 7, 10, 12, 15, 20])
# # }

# search_space = {
#     "lr": tune.choice([1e-7, 2e-7, 5e-7, 1e-6, 5e-6, 1e-5, 2e-5]),
#     "batch_size": tune.choice([4, 8]),
#     "epochs": tune.choice([5, 10, 15])
# }


# # reporter = CLIReporter(
# #     metric_columns=["loss", "training_iteration", "train_loss", "similarity", "early_stopping_epoch"],
# #     print_intermediate_tables=True,
# # )
    
# reporter = JupyterNotebookReporter(
#     metric_columns=["loss", "training_iteration", "train_loss", "similarity", "early_stopping_epoch"],
#     print_intermediate_tables=False,
# )
    
# optuna_search = OptunaSearch(
#     metric="loss",
#     mode="min"
# )
    
    

# scheduler = ASHAScheduler(
#     metric='loss',
#     mode='min',
#     max_t=16,
#     grace_period=3,
#     reduction_factor=2
# )



# analysis = tune.run(
#     train_fn,
#     config=search_space,
#     num_samples=15,
#     resources_per_trial={"cpu": 2, "gpu": 1},
#     progress_reporter=reporter,
#     search_alg=optuna_search
# )

# # print("Best hyperparameters found:", analysis.best_config)
    
# best_trial = analysis.get_best_trial("loss", "min", "last")

# print(f"Best trial config: {best_trial.config}")
# print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
# print(f"Best trial final training loss: {best_trial.last_result['train_loss']}")
# print(f"Best trial final cosine similarity score: {best_trial.last_result['similarity']}")
# print(f"Best trial final Early Stopping Epoch: {best_trial.last_result['early_stopping_epoch']}")

In [28]:
# Function to train the model with best hyperparameters
def train_best_model(config, device, train_dataset = train_ds_pd, val_dataset = validation_ds_pd):
    
    print(f"""
    Tuning Model with:
    {config}
    """)
    
    best_val_loss = float("inf")
    best_model_path = None
    epochs_since_improvement = 0
    
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

    model.to(device)
    
    train_dataset = TextDataset(tokenizer, train_dataset)
    val_dataset = TextDataset(tokenizer, val_dataset)
    
    train_loader = DataLoader(train_dataset, batch_size=int(config['batch_size']), shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=int(config['batch_size']), shuffle=False)

    optimizer = AdamW(model.parameters(), lr=config['lr'])
    scaler = GradScaler()  # Mixed precision scaler
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
    patience=5
    early_stopping = EarlyStopping(patience, verbose=True)

    for epoch in range(int(config['epochs'])+patience+1):
        model.train()
        total_train_loss = 0.0
        
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            optimizer.zero_grad()
            
            with autocast(device_type=device.type):
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_train_loss += loss.item()
            
        avg_train_loss = total_train_loss / len(train_loader)

        model.eval()
        total_val_loss = 0.0
        val_similarity = 0.0
        num_val_samples = 0
        
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                with autocast(device_type=device.type):
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss

                preds = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=512)
                pred_texts = tokenizer.batch_decode(preds, skip_special_tokens=True)
                target_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

                for pred_text, target_text in zip(pred_texts, target_texts):
                    similarity = calculate_cosine_similarity(pred_text, target_text, model, tokenizer, device)
                    val_similarity += similarity
                    num_val_samples += 1

                total_val_loss += loss.item()
                
        avg_val_loss = total_val_loss / len(val_loader)
#         avg_val_similarity = val_similarity / len(val_loader)
        avg_val_similarity = val_similarity / num_val_samples
        
        checkpoint_path = f"/kaggle/working/best_checkpoint.pt"
        
        scheduler.step(avg_val_loss)
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_since_improvement = 0
            best_model_path = checkpoint_path
            # Save the model checkpoint with the best validation loss
#             checkpoint_path = f"/kaggle/working/best_checkpoint.pt"
#             torch.save(model.state_dict(), checkpoint_path)
            torch.save({
                'model_state_dict': model.state_dict(),
                'config': config  # Save configuration
            }, checkpoint_path)
    
        else:
            epochs_since_improvement += 1

        early_stopping(avg_val_loss, model)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            print(f"Best Model Epoch Saved: {epoch - patience+1}")
#             print("Early stopping")
            break

        print(f"""
        Validation Loss: {avg_val_loss},
        Training Loss: {avg_train_loss},
        Cosine Similarity: {avg_val_similarity},
        Epochs: {epoch+1}
        """)
            
    print(f"Best Validation Loss: {best_val_loss}")

    # Save the model
#     model.save_pretrained('best_model')
#     tokenizer.save_pretrained('best_model_tokenizer')

    return model, best_model_path

In [29]:
# model_path = 'best_model'
# best_config = analysis.best_config

# best_config = best_trial.config

default_config = {'lr': 1e-06, 'batch_size': 8, 'epochs': 15}

# best_config = best_trial.config

try:
    best_trial = analysis.get_best_trial("loss", "min", "last")
except NameError:
    best_trial = None  # Ensure best_trial is always defined

# Check if best_trial exists and has a valid config, otherwise use default
if best_trial and hasattr(best_trial, "config") and best_trial.config:
    best_config = best_trial.config
else:
    best_config = default_config
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


best_model, best_model_path = train_best_model(best_config, device, train_ds_pd, validation_ds_pd)


    Tuning Model with:
    {'lr': 1e-06, 'batch_size': 8, 'epochs': 15}
    


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



        Validation Loss: 2.2676671476506476,
        Training Loss: 6.640102089486188,
        Cosine Similarity: 0.9097542115268743,
        Epochs: 1
        

        Validation Loss: 0.5576981177080923,
        Training Loss: 1.7225606621036098,
        Cosine Similarity: 0.9196197973742494,
        Epochs: 2
        

        Validation Loss: 0.1862004178562271,
        Training Loss: 0.503743706167572,
        Cosine Similarity: 0.9296142846960959,
        Epochs: 3
        

        Validation Loss: 0.10474627253724568,
        Training Loss: 0.21226360940555525,
        Cosine Similarity: 0.9390722166530187,
        Epochs: 4
        

        Validation Loss: 0.07582745277114324,
        Training Loss: 0.12700913608164732,
        Cosine Similarity: 0.9444924544885503,
        Epochs: 5
        

        Validation Loss: 0.062337606262638054,
        Training Loss: 0.09183506160398894,
        Cosine Similarity: 0.9487238854002102,
        Epochs: 6
        

        Validati

In [30]:
final_model_path = "/kaggle/working/final_best_model.pt"
# torch.save(best_model.state_dict(), final_model_path)
torch.save({'model_state_dict': best_model.state_dict(),'config': best_config}, final_model_path)

print(f"Best model saved at: {final_model_path}")

Best model saved at: /kaggle/working/final_best_model.pt


In [31]:


# Load the best model
final_model_path = "/kaggle/working/final_best_model.pt"
# final_model_path = "/kaggle/input/tachygraphy-lv2-emotion-moodtags-v1/transformers/default/1/final_best_model.pt"


checkpoint = torch.load(final_model_path)
# config = checkpoint['config']

# loaded_model = SentimentModel(
#     roberta_model=RobertaModel.from_pretrained("roberta-base"),
#     n_classes=3,
#     dropout_rate=config["dropout_rate"]
# )
# loaded_model = DisasterModel(
#     roberta_model=DebertaV2Model.from_pretrained('microsoft/deberta-v3-base'),
#     n_classes=2,
#     dropout_rate=config["dropout_rate"]
# )
loaded_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_model = loaded_model.to(device)
loaded_model.eval()

print(f"Loaded model from {final_model_path} for inference.")



<ipython-input-31-fdf5a391eb5e>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(final_model_path)


Loaded model from /kaggle/working/final_best_model.pt for inference.


In [32]:
# def predict(model, tokenizer, input_texts, device):
# #     model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
# #     model.load_state_dict(torch.load(model_path)['model_state_dict'])
#     model.to(device)
# #     model.eval()

#     predictions = []
#     with torch.no_grad():
#         for text in input_texts:
#             encodings = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=256).to(device)
#             input_ids = encodings['input_ids']
#             attention_mask = encodings['attention_mask']
            
#             preds = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=256)
#             pred_text = tokenizer.decode(preds[0], skip_special_tokens=True)
#             predictions.append(pred_text)
    
#     return predictions

In [33]:
# # Predict function for random input
# def predict_on_random_input(model, tokenizer, input_text):
#     model.eval()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
#     inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)
    
#     with torch.no_grad():
#         preds = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=128)
    
#     pred_text = tokenizer.decode(preds[0], skip_special_tokens=True)
#     return pred_text

In [34]:
def predict(model, tokenizer, input_texts, device):
#     model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
#     model.load_state_dict(torch.load(model_path)['model_state_dict'])
    model.to(device)
#     model.eval()

    predictions = []
    with torch.no_grad():
#         for text in input_texts:
        text = input_texts
        encodings = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=256).to(device)
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']
            
        preds = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=512)
        pred_text = tokenizer.decode(preds[0], skip_special_tokens=True)
        predictions.append(pred_text)
    
    return predictions

In [35]:
# Test prediction on random input
random_input = "pls hlp, idk wht 2 do"
predicted_text = predict(loaded_model, tokenizer, random_input, device)
print(f"Input: {random_input}")
print(f"Predicted Meaning: {predicted_text}")

Input: pls hlp, idk wht 2 do
Predicted Meaning: ["Please help, I don't know what to do."]


In [36]:
# Test prediction on random input
random_input = "i don8 no fr y hes soooo sad"
predicted_text = predict(loaded_model, tokenizer, random_input, device)
print(f"Input: {random_input}")
print(f"Predicted Meaning: {predicted_text}")

Input: i don8 no fr y hes soooo sad
Predicted Meaning: ["i don't know why he's so sad."]


In [37]:
random_input = "hey! how are you, wanna ply valo toni8?"
predicted_text = predict(loaded_model, tokenizer, random_input, device)
print(f"Input: {random_input}")
print(f"Predicted Meaning: {predicted_text}")

Input: hey! how are you, wanna ply valo toni8?
Predicted Meaning: ['Hey! how are you, want to ply valo tonight?']
